###### Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

In [2]:
### Dataset Directories

trainDir = 'datasets/train/'
testDir = 'datasets/test/'
valDir = 'datasets/validation/'

###### Data Preprocessing

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
trainDataGen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

testDataGen = ImageDataGenerator(rescale=1./255)

trainGenerator = trainDataGen.flow_from_directory(
    trainDir,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

valGenerator = testDataGen.flow_from_directory(
    valDir,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


###### Using Convolution Base of VGG16

In [7]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 13s 0us/step


In [8]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [12]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [14]:
from tensorflow.keras import optimizers

In [17]:
model.compile(loss = 'binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

In [18]:
checkpoint = keras.callbacks.ModelCheckpoint('CNN-Model-{epoch:02d}.h5', save_best_only=True)

history = model.fit(
    trainGenerator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=valGenerator,
    validation_steps=50,
    callbacks=[checkpoint]
)

Epoch 1/30
  6/100 [>.............................] - ETA: 21:51 - loss: 0.7412 - acc: 0.4763

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,7))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
histDF = pd.DataFrame(history.history)

In [ ]:
histCSV = 'history.csv'
with open(histCSV, mode='w') as f:
    histDF.to_csv(f)

In [19]:
testGenerator = testDataGen.flow_from_directory(
    testDir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [ ]:
model.evaluate_generator(testGenerator, steps=50)